## 점포 data

In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [4]:
store = pd.read_csv(
    filepath_or_buffer = "../data/점포.csv", 
    encoding = "cp949"
    )
store.rename(columns={'소재지':'주소', '상호명':'시설명'}, inplace=True)
store

,연번,기초지자체명,구분1,구분2,시설명,개설등록일,주소,매장면적(제곱미터)
0,1,창원시,대규모점포,백화점,대동백화점,1995-05-01,경남 창원시 성산구 동산로 115,8123.0
1,2,창원시,대규모점포,백화점,㈜신세계 마산점,2001-03-26,경남 창원시 마산합포구 합포로 251(산호동),26675.0
2,3,창원시,대규모점포,백화점,롯데백화점 창원점,2001-12-24,(51494) 경남 창원시 성산구 중앙대로 124(상남동),27498.0
3,4,창원시,대규모점포,백화점,롯데백화점 마산점,2008-11-26,경남 창원시 마산합포구 동서동로 18(신포동2가),29244.0
4,5,창원시,대규모점포,백화점,롯데백화점 창원점 신관,2012-12-20,경남 창원시 성산구 상남로 121(상남동),20536.0
...,...,...,...,...,...,...,...,...
62,63,양산시,대규모점포,전문점,모다아울렛,2013-08-23,양산시 중부로 2 (중부동),16634.0
63,64,양산시,대규모점포,대형마트,이마트 트레이더스 양산점,2013-04-19,양산시 평산로 16 (평산동),8027.0
64,65,양산시,대규모점포,대형마트,롯데마트 웅상점,2004-12-24,양산시 삼호1길 34 (삼호동),8595.0
65,66,양산시,대규모점포,대형마트,이마트 양산점,2004-06-16,양산시 양산역6길 12(중부동),10936.0


In [5]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in store.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({ "경도": x, "위도": y})
        else:
            coordinates.append({ "경도": None, "위도": None})
    else:
        coordinates.append({ "경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 확인
coord_df

,경도,위도
0,128.690472329,35.219180383
1,128.586388498,35.219971839
2,128.681842013,35.224560386
3,128.573438520,35.201517837
4,128.682881552,35.224202273
...,...,...
62,129.027462559,35.338994211
63,129.152757059,35.382143149
64,129.166964473,35.412569889
65,129.026492546,35.336842105


In [6]:
# 데이터프레임 합치기
store = pd.concat([store, coord_df], axis=1)
store

,연번,기초지자체명,구분1,구분2,시설명,개설등록일,주소,매장면적(제곱미터),경도,위도
0,1,창원시,대규모점포,백화점,대동백화점,1995-05-01,경남 창원시 성산구 동산로 115,8123.0,128.690472329,35.219180383
1,2,창원시,대규모점포,백화점,㈜신세계 마산점,2001-03-26,경남 창원시 마산합포구 합포로 251(산호동),26675.0,128.586388498,35.219971839
2,3,창원시,대규모점포,백화점,롯데백화점 창원점,2001-12-24,(51494) 경남 창원시 성산구 중앙대로 124(상남동),27498.0,128.681842013,35.224560386
3,4,창원시,대규모점포,백화점,롯데백화점 마산점,2008-11-26,경남 창원시 마산합포구 동서동로 18(신포동2가),29244.0,128.573438520,35.201517837
4,5,창원시,대규모점포,백화점,롯데백화점 창원점 신관,2012-12-20,경남 창원시 성산구 상남로 121(상남동),20536.0,128.682881552,35.224202273
...,...,...,...,...,...,...,...,...,...,...
62,63,양산시,대규모점포,전문점,모다아울렛,2013-08-23,양산시 중부로 2 (중부동),16634.0,129.027462559,35.338994211
63,64,양산시,대규모점포,대형마트,이마트 트레이더스 양산점,2013-04-19,양산시 평산로 16 (평산동),8027.0,129.152757059,35.382143149
64,65,양산시,대규모점포,대형마트,롯데마트 웅상점,2004-12-24,양산시 삼호1길 34 (삼호동),8595.0,129.166964473,35.412569889
65,66,양산시,대규모점포,대형마트,이마트 양산점,2004-06-16,양산시 양산역6길 12(중부동),10936.0,129.026492546,35.336842105


In [7]:
store = store[['시설명','주소','경도','위도']]
store

,시설명,주소,경도,위도
0,대동백화점,경남 창원시 성산구 동산로 115,128.690472329,35.219180383
1,㈜신세계 마산점,경남 창원시 마산합포구 합포로 251(산호동),128.586388498,35.219971839
2,롯데백화점 창원점,(51494) 경남 창원시 성산구 중앙대로 124(상남동),128.681842013,35.224560386
3,롯데백화점 마산점,경남 창원시 마산합포구 동서동로 18(신포동2가),128.573438520,35.201517837
4,롯데백화점 창원점 신관,경남 창원시 성산구 상남로 121(상남동),128.682881552,35.224202273
...,...,...,...,...
62,모다아울렛,양산시 중부로 2 (중부동),129.027462559,35.338994211
63,이마트 트레이더스 양산점,양산시 평산로 16 (평산동),129.152757059,35.382143149
64,롯데마트 웅상점,양산시 삼호1길 34 (삼호동),129.166964473,35.412569889
65,이마트 양산점,양산시 양산역6길 12(중부동),129.026492546,35.336842105


## 전통시장 데이터

In [15]:
traditional_market = pd.read_csv(
    filepath_or_buffer = "../data/전통시장 데이터.csv", 
    encoding = "cp949"
    )
traditional_market = traditional_market[traditional_market["소재지 도로명주소"].str.contains("경상남도")].reset_index(drop=True)
traditional_market.rename(columns={'소재지 지번 주소':'주소', '시장명':'시설명'}, inplace=True)
traditional_market

,시장코드,시설명,시장 유형,주소,소재지 도로명주소,아케이드 보유 여부,엘리베이터/에스컬레이터 보유여부,고객지원센터 보유 여부,스프링쿨러 보유 여부,화재감지기 보유여부,...,쇼핑카트_보유여부,외국인 안내센터_보유여부,고객동선통로_보유여부,방송센터_보유여부,문화교실_보유여부,공동물류창고_보유여부,시장전용 고객주차장_보유여부,교육장_보유여부,회의실_보유여부,자동심장충격기_보유여부
0,a42020002,산성뿌리전통시장,전통시장,경상남도 창원시 마산합포구 문화동15길 13,경상남도 창원시 마산합포구 해운동 64,Y,N,Y,N,Y,...,Y,N,Y,Y,Y,N,Y,Y,Y,N
1,a41060002,강경젓갈시장,전통시장,경상남도 함양군 함양읍 용평 중앙길 20,경상남도 함양군 함양읍 용평리 607-104,Y,N,Y,N,N,...,N,N,N,N,N,N,N,Y,Y,N
2,a41390002,당진전통시장,전통시장,경상남도 양산시 중앙로 232,경상남도 양산시 북부동 148-19,Y,N,Y,Y,Y,...,Y,N,Y,Y,N,Y,Y,Y,Y,Y
3,a41030011,대천항종합수산물시장,전통시장,경상남도 창원시 마산 회원구 합성남8길 76,경상남도 창원시 마산 회원구 합성동 326-1,Y,Y,Y,Y,Y,...,N,N,Y,Y,N,N,Y,Y,Y,N
4,a43360001,괴산전통시장,전통시장,경상남도 창원시 성산구 상남로 71,경상남도 창원시 성산구 상남동 73-4,Y,N,Y,Y,Y,...,N,N,Y,Y,Y,Y,Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,a33050002,장성중앙시장,전통시장,경상남도 하동군 경서대로 2427-14,경상남도 하동군 북천면 직전리 597-1,Y,N,Y,N,Y,...,Y,N,N,Y,N,Y,Y,Y,Y,N
102,a33310002,홍천시장,전통시장,경상남도 하동군 횡천면 문화2길 24,경상남도 하동군 횡천면 횡천리 507-2,Y,N,N,Y,Y,...,Y,N,Y,Y,N,N,Y,N,N,N
103,a64010004,제주시동문재래시장,전통시장,경상남도 진주시 대곡면 진의로 1079번길 6,경상남도 진주시 대곡면 광석리 321,Y,N,Y,Y,Y,...,Y,Y,Y,Y,Y,N,Y,Y,Y,N
104,a64010002,동문시장(주),전통시장,경상남도 함양군 마천면 천왕봉로 1144-2,경상남도 함양군 마천면 가흥리 609-2,Y,N,N,N,Y,...,N,N,Y,Y,N,N,N,Y,Y,N


In [16]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in traditional_market.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({ "경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 확인
coord_df


,경도,위도
0,128.561716679,35.181222908
1,127.728988668,35.518603995
2,129.041196176,35.350450402
3,128.587663460,35.240220183
4,128.679541596,35.220442395
...,...,...
101,127.891709103,35.113546527
102,127.810894449,35.109159231
103,128.168556002,35.263701105
104,127.662681215,35.395649333


In [17]:
# 데이터프레임 합치기
traditional_market = pd.concat([traditional_market, coord_df], axis=1)
traditional_market

,시장코드,시설명,시장 유형,주소,소재지 도로명주소,아케이드 보유 여부,엘리베이터/에스컬레이터 보유여부,고객지원센터 보유 여부,스프링쿨러 보유 여부,화재감지기 보유여부,...,고객동선통로_보유여부,방송센터_보유여부,문화교실_보유여부,공동물류창고_보유여부,시장전용 고객주차장_보유여부,교육장_보유여부,회의실_보유여부,자동심장충격기_보유여부,경도,위도
0,a42020002,산성뿌리전통시장,전통시장,경상남도 창원시 마산합포구 문화동15길 13,경상남도 창원시 마산합포구 해운동 64,Y,N,Y,N,Y,...,Y,Y,Y,N,Y,Y,Y,N,128.561716679,35.181222908
1,a41060002,강경젓갈시장,전통시장,경상남도 함양군 함양읍 용평 중앙길 20,경상남도 함양군 함양읍 용평리 607-104,Y,N,Y,N,N,...,N,N,N,N,N,Y,Y,N,127.728988668,35.518603995
2,a41390002,당진전통시장,전통시장,경상남도 양산시 중앙로 232,경상남도 양산시 북부동 148-19,Y,N,Y,Y,Y,...,Y,Y,N,Y,Y,Y,Y,Y,129.041196176,35.350450402
3,a41030011,대천항종합수산물시장,전통시장,경상남도 창원시 마산 회원구 합성남8길 76,경상남도 창원시 마산 회원구 합성동 326-1,Y,Y,Y,Y,Y,...,Y,Y,N,N,Y,Y,Y,N,128.587663460,35.240220183
4,a43360001,괴산전통시장,전통시장,경상남도 창원시 성산구 상남로 71,경상남도 창원시 성산구 상남동 73-4,Y,N,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,128.679541596,35.220442395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,a33050002,장성중앙시장,전통시장,경상남도 하동군 경서대로 2427-14,경상남도 하동군 북천면 직전리 597-1,Y,N,Y,N,Y,...,N,Y,N,Y,Y,Y,Y,N,127.891709103,35.113546527
102,a33310002,홍천시장,전통시장,경상남도 하동군 횡천면 문화2길 24,경상남도 하동군 횡천면 횡천리 507-2,Y,N,N,Y,Y,...,Y,Y,N,N,Y,N,N,N,127.810894449,35.109159231
103,a64010004,제주시동문재래시장,전통시장,경상남도 진주시 대곡면 진의로 1079번길 6,경상남도 진주시 대곡면 광석리 321,Y,N,Y,Y,Y,...,Y,Y,Y,N,Y,Y,Y,N,128.168556002,35.263701105
104,a64010002,동문시장(주),전통시장,경상남도 함양군 마천면 천왕봉로 1144-2,경상남도 함양군 마천면 가흥리 609-2,Y,N,N,N,Y,...,Y,Y,N,N,N,Y,Y,N,127.662681215,35.395649333


In [18]:
traditional_market = traditional_market[['시설명','주소','경도','위도']]
traditional_market

,시설명,주소,경도,위도
0,산성뿌리전통시장,경상남도 창원시 마산합포구 문화동15길 13,128.561716679,35.181222908
1,강경젓갈시장,경상남도 함양군 함양읍 용평 중앙길 20,127.728988668,35.518603995
2,당진전통시장,경상남도 양산시 중앙로 232,129.041196176,35.350450402
3,대천항종합수산물시장,경상남도 창원시 마산 회원구 합성남8길 76,128.587663460,35.240220183
4,괴산전통시장,경상남도 창원시 성산구 상남로 71,128.679541596,35.220442395
...,...,...,...,...
101,장성중앙시장,경상남도 하동군 경서대로 2427-14,127.891709103,35.113546527
102,홍천시장,경상남도 하동군 횡천면 문화2길 24,127.810894449,35.109159231
103,제주시동문재래시장,경상남도 진주시 대곡면 진의로 1079번길 6,128.168556002,35.263701105
104,동문시장(주),경상남도 함양군 마천면 천왕봉로 1144-2,127.662681215,35.395649333


In [19]:
sales = pd.concat([store, traditional_market], axis=0).reset_index(drop=[True])
sales

,시설명,주소,경도,위도
0,대동백화점,경남 창원시 성산구 동산로 115,128.690472329,35.219180383
1,㈜신세계 마산점,경남 창원시 마산합포구 합포로 251(산호동),128.586388498,35.219971839
2,롯데백화점 창원점,(51494) 경남 창원시 성산구 중앙대로 124(상남동),128.681842013,35.224560386
3,롯데백화점 마산점,경남 창원시 마산합포구 동서동로 18(신포동2가),128.573438520,35.201517837
4,롯데백화점 창원점 신관,경남 창원시 성산구 상남로 121(상남동),128.682881552,35.224202273
...,...,...,...,...
168,장성중앙시장,경상남도 하동군 경서대로 2427-14,127.891709103,35.113546527
169,홍천시장,경상남도 하동군 횡천면 문화2길 24,127.810894449,35.109159231
170,제주시동문재래시장,경상남도 진주시 대곡면 진의로 1079번길 6,128.168556002,35.263701105
171,동문시장(주),경상남도 함양군 마천면 천왕봉로 1144-2,127.662681215,35.395649333


In [20]:
sales.to_csv("../data/판매시설.csv", index=False)